<a href="https://colab.research.google.com/github/Lancer59/BigDataAnalysis/blob/main/Assignment_(RDD%2C_DataFrame%2C_ML).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Question 1 on RDD**

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 50 kB/s 
     |████████████████████████████████| 199 kB 60.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=9490a027d8976f86dd5883f4a106ac80ca386ad536590d7430ff3f2d6547d23b
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [3]:
#store file as rdd.
from pyspark import SparkContext
sc=SparkContext.getOrCreate()
rdd=sc.textFile('/content/lvl.txt')

#Remove blankspaces.
srdd=rdd.filter(lambda l:l.strip())

#Split into words.
wrdd=srdd.flatMap(lambda l:l.split(" "))

#Remove stopwords.
stopwords=["in","on","for","the","a"]
stpwrd=wrdd.filter(lambda w:w not in stopwords)


#Remove punctuations.
import string
rrdd=stpwrd.map(lambda w:w.translate(str.maketrans('','',string.punctuation)))

##(i) Word with highest number of occurrence.
grdd=wrdd.groupBy(lambda w:w[:])
mrdd=grdd.map(lambda x:(len(x[1]),x[0]))
srdd=mrdd.sortByKey(False)
hnw=srdd.take(2)[0][1]
print("\n(i) The word with highest number of occurrence = "+str(hnw))

##(ii) Character with highest number of occurrence.
grdd=wrdd.groupBy(lambda w:w[0:1])
mrdd=grdd.map(lambda x:(len(x[1]),x[0]))
srdd=mrdd.sortByKey(False)
hnc=srdd.take(2)[0][1]
print("\n(ii) The character with highest number of occurrence = "+str(hnc))

##(iii) Compute occurrence of any given word accepted from keyboard.
we=input("\n(iii) Enter the word of whose occurrence must be returned: ")
grdd=wrdd.groupBy(lambda w:w[:])
mrdd=grdd.map(lambda x:(len(x[1]),x[0]))
swrdd=mrdd.filter(lambda w:w[1] == we)
wno=swrdd.take(1)[0][0]
print("      The occurrence of the entered word is = "+str(wno))

##(iv) Line sorted in descending order of number of words they contain.
mrdd=rdd.map(lambda x:(len(x.split(" ")),x))
srdd=mrdd.sortByKey(False)
wind=list(srdd.take(srdd.count()))
print("\n(iv) The Lines sorted in descending order of number of words they contain: \n")
for i in wind:
  print(i[1])

##(v) Take 80% sample twice and perform intersection between them and check number of lines common.
srdd1=rrdd.sample(False, 0.8, 10)
srdd2=rrdd.sample(False, 0.8, 90)
isample=srdd1.intersection(srdd2)
print("\n (v)")
print(" The number of lines in sample 1: "+str(srdd1.count()))
print(" The number of lines in sample 2: "+str(srdd2.count()))
print(" The number of lnes which is common: "+str(isample.count()))











(i) The word with highest number of occurrence = I

(ii) The character with highest number of occurrence = a

(iii) Enter the word of whose occurrence must be returned: the
      The occurrence of the entered word is = 16

(iv) The Lines sorted in descending order of number of words they contain: 

I love you so much and I am so glad I met you. You are the one I have been waiting for all of my life. The little things, like making sure to let me know when I look beautiful and holding my hand whenever you can, makes me feel so loved. You are the love of my life. Without you I am incomplete. I love you more than any woman I have ever met or ever will meet in the future. You are amazing
Every time I look at you, my heart skips a beat. When I feel your hugs, my strength becomes tenfold. I cannot imagine a day without you — and am certain I don’t want to know what that feels like… I love you more than words can even say; you are the light of my life, my soulmate, and my best friend.
You are

# **Question 2 on DataFrames**

In [32]:
#dataframe construction
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col
sess=SparkSession.builder.getOrCreate()
df=sess.read.csv('/content/cars.csv',header=True) 

##(i) Find the number of records whose income > 8000
A1=df.filter(df.income>8000).count()
print("\n (i) The number of records whose income > 8000 is = "+str(A1))

##(ii) Find the number of records whose income <4000 and sales>1000
A21=df.filter(df.income<4000)
A2=A21.filter(df.sales>1000).count()
print("\n (ii) The number of records whose income <4000 and sales>1000 is = "+str(A2))

##(iii) Find the number of records for different ages and order them in descending order
A31=df.select('age').distinct()
print("\n (iii) The number of records for different ages is = "+str(A31.count())+"\n The order:")
A3=df.select('age').groupBy('age').count().orderBy(col('count').desc())
print(str(A3.show()))

##(iv) Find mean sales of male(1) and female (0)
A4=df.groupBy('gender').agg({'sales':'Mean'})
print("\n (iv) Mean sales of male(1) and female(0) are: ")
A4.show()

##(v) Find the descriptive statistics of debt
A5=df.select('debt').describe()
print("\n (v) descriptive statistics of debt: ")
A5.show()


##(vi) Find correlation between miles and debt
import pyspark.sql.types as typ
from pyspark.sql.session import SparkSession
from pyspark import SparkContext,SQLContext
labels=[
    ('age',typ.IntegerType()),
    ('gender',typ.IntegerType()),
    ('miles',typ.IntegerType()),
    ('debt',typ.IntegerType()),
    ('income',typ.IntegerType()),
    ('sales',typ.IntegerType())
]
schema = typ.StructType([
typ.StructField(e[0], e[1], False) for e in labels
])
sess=SparkSession.builder.getOrCreate()
df=sess.read.csv('/content/cars.csv',header=True,schema=schema) 
print("\n (vi) Correlation between miles and debt: ")
A6=df.stat.corr('miles','debt')
print(str(A6))

##(vii)Convert the dataframe to table and write an SQL query for finding
       #grouping of age,gender and number of records of each group
print("\n (vii) Age,gender and number of records of each group: ")
df.createOrReplaceTempView('cars_table')
sq=SQLContext(sc)
sq.sql('select age,gender,count(*) from cars_table group by age,gender ').show()

###(viii)Store the results of question (iii) in a csv called A8.csv
print("\n (viii) Results of question (iii) in a csv called A8! ")
A8=df.groupBy('age').count().orderBy(col('count').desc())
A8.write.format("csv").save('/content/A8.csv')



 (i) The number of records whose income > 8000 is = 295

 (ii) The number of records whose income <4000 and sales>1000 is = 235

 (iii) The number of records for different ages is = 42
 The order:
+---+-----+
|age|count|
+---+-----+
| 25|   41|
| 30|   34|
| 27|   33|
| 24|   32|
| 37|   32|
| 36|   30|
| 28|   30|
| 29|   29|
| 21|   29|
| 26|   29|
| 20|   28|
| 22|   28|
| 23|   25|
| 49|   24|
| 40|   24|
| 45|   24|
| 46|   23|
| 58|   23|
| 48|   23|
| 59|   23|
+---+-----+
only showing top 20 rows

None

 (iv) Mean sales of male(1) and female(0) are: 
+------+------------------+
|gender|        avg(sales)|
+------+------------------+
|     0|12024.953091684434|
|     1|11371.726720647774|
+------+------------------+


 (v) descriptive statistics of debt: 
+-------+------------------+
|summary|              debt|
+-------+------------------+
|  count|               963|
|   mean|14109.004153686397|
| stddev| 18273.70248104874|
|    min|                 0|
|    max|              

# **Question 3 on Machine Learning**

In [47]:
from numpy.core.numeric import array_equiv
##(i) Data Preprocessing
from pyspark.sql.session import SparkSession
sess=SparkSession.builder.getOrCreate()
import pyspark.sql.types as typ
labels=[('PM25',typ.DoubleType()),
        ('PM10',typ.DoubleType()),
        ('NO',typ.DoubleType()),
        ('NO2',typ.DoubleType()),
        ('NOX',typ.DoubleType()),
        ('NH3',typ.DoubleType()),
        ('CO',typ.DoubleType()),
        ('SO2',typ.DoubleType()),
        ('O3',typ.DoubleType()),
        ('Benzene',typ.DoubleType()),
        ('Toluene',typ.DoubleType()),
        ('Xylene',typ.DoubleType()),
        ('AQI',typ.DoubleType()),
       
        ]
schema = typ.StructType([
typ.StructField(e[0], e[1], False) for e in labels
])
aq=sess.read.csv("/content/airqualitydataset.csv",header=True,schema=schema)
aq.dropDuplicates()
aq.dropna()

##(ii) Filter our important columns contributing to AQI
import pyspark.mllib.stat as st 
from pyspark.ml.feature import VectorAssembler
rdd = aq.rdd 
rdd.take(2)
rdd=rdd.map(lambda row: [e for e in row])
rdd.take(2)
import numpy as np
sta = st.Statistics.colStats(rdd)
sta.mean(),np.sqrt(sta.variance())
print("\n Correlation corresponding to last AQI column: ")
corrs = st.Statistics.corr(rdd) 
print(str(corrs[-1]))
print("\n AirQuality columns: ")
print(str(aq.columns))
aq=aq.select('PM25','PM10','NO2','NOX','SO2','aqi')
vectorAssembler = VectorAssembler(inputCols = [ 'PM25','PM10','NO2','NOX','SO2',], outputCol = 'features')
aqv = vectorAssembler.transform(aq)
print("\n The filtered features and aqi columns: " )
aqv.select('features','aqi').show()

##(iii) Split into 70, 30 train and test dataframes
aqs = aqv.randomSplit([0.7, 0.3])
train=aqs[0]
test=aqs[1]
print("\n The Train and Test count respectively "+str(train.count())+" "+str(test.count()))

##(iv) Apply Linear Regression, Decision Tree classifier and Gradient Boost
       #models on train data frames

#Linear regression
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol = 'features', labelCol='aqi', maxIter=10, regParam=0.3, elasticNetParam=0.8)#balance between overfitting and underfitting
lrm = lr.fit(train)
#Weights
print("\n Coefficients: " + str(lrm.coefficients))
#Bias
print(" Intercept: " + str(lrm.intercept))
print()
#Decision Tree Classifier
from pyspark.ml.regression import DecisionTreeRegressor

dt = DecisionTreeRegressor(featuresCol ='features', labelCol = 'aqi')
dtm = dt.fit(train)

#Gradient Booster
from pyspark.ml.regression import GBTRegressor

gbt = GBTRegressor(featuresCol = 'features', labelCol = 'aqi', maxIter=10)
gbtm = gbt.fit(train)

##(v)For each model find predictions on test dataframes and evaluate
    #prediction accuracy.

#Linear Regression
lrp = lrm.transform(test)
lrp.select("prediction","aqi","features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lre = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="aqi",metricName="r2")
print("\n The accuracy for Linear regression is: "+str(lre.evaluate(lrp)))

#Decision tree classifier
dtp = dtm.transform(test)
dte = RegressionEvaluator(
labelCol="aqi", predictionCol="prediction", metricName="r2")
r2 = dte.evaluate(dt_predictions)
print("\n The accuracy for Decision tree classifier is: "+str(r2))

#Gradient booster
gbtp = gbtm.transform(test)
gbte = RegressionEvaluator(
labelCol="aqi", predictionCol="prediction", metricName="r2")
r2 = gbte.evaluate(gbtp)
print("\n The accuracy for Gradient Booster is: "+str(r2))




 Correlation corresponding to last AQI column: 
[0.91519013 0.90473294 0.47765931 0.54618293 0.58023652 0.34522906
 0.60818697 0.18335878 0.09487005 0.14251333 0.21557067 0.14224832
 1.        ]

 AirQuality columns: 
['PM25', 'PM10', 'NO', 'NO2', 'NOX', 'NH3', 'CO', 'SO2', 'O3', 'Benzene', 'Toluene', 'Xylene', 'AQI']

 The filtered features and aqi columns: 
+--------------------+-----+
|            features|  aqi|
+--------------------+-----+
|[81.4,124.5,20.5,...|184.0|
|[78.32,129.06,26....|197.0|
|[88.76,135.32,30....|198.0|
|[64.18,104.09,28....|188.0|
|[72.47,114.84,23....|173.0|
|[69.8,114.86,20.1...|165.0|
|[73.96,113.56,19....|191.0|
|[89.9,140.2,26.19...|191.0|
|[87.14,130.52,21....|227.0|
|[84.64,125.0,26.9...|168.0|
|[88.36,121.77,20....|198.0|
|[96.83,139.36,25....|201.0|
|[117.46,181.64,41...|252.0|
|[122.88,208.86,54...|310.0|
|[74.28,141.22,44....|196.0|
|[50.32,102.77,33....|132.0|
|[58.47,115.27,41....|147.0|
|[89.35,131.48,42....|179.0|
|[64.42,99.74,34.7...|145.0|